In [1]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
if True:
    max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
    dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00


In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/')
async def root():
    return {'app': 'mistral-7b finetuning with user prompts and JSON'}

@app.get('/run')
async def run_model(user_prompt):
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Generate json data from the given user prompt. User prompt is similar to sql queries", # instruction
            f"{user_prompt}", # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    output = tokenizer.batch_decode(outputs)
    return output[0][output[0].find('Response:\n'):].replace('Response:\n', '').replace('</s>', '').strip()

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

In [ ]:
# Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
auth_token = "2f3gCdn7HVE84fN7sCExUyEYyyh_2cDFFtWPDUe7yWwfokdMP"

# Set the authtoken
ngrok.set_auth_token(auth_token)

# Connect to ngrok
ngrok_tunnel = ngrok.connect(8000)

# Print the public URL
print('Public URL:', str(ngrok_tunnel.public_url)+'/docs')

# Apply nest_asyncio
nest_asyncio.apply()

# Run the uvicorn server
uvicorn.run(app, port=8000)

INFO:     Started server process [4568]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://1ed7-34-124-217-128.ngrok-free.app
INFO:     34.205.177.101:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2600:8806:2601:a300:10f5:7db2:c34f:49fc:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2600:8806:2601:a300:10f5:7db2:c34f:49fc:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2600:8806:2601:a300:10f5:7db2:c34f:49fc:0 - "GET / HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     2600:8806:2601:a300:10f5:7db2:c34f:49fc:0 - "GET /run?user_prompt=Replace%20the%20contents%20of%20%22RICCode%22%20with%20the%20contents%20of%20%22Sedol%22 HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

INFO:     2600:8806:2601:a300:10f5:7db2:c34f:49fc:0 - "GET /run?user_prompt=add%20structure%20to%20comma%20delimited%20file HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     2600:8806:2601:a300:10f5:7db2:c34f:49fc:0 - "GET /run?user_prompt=Filter%20where%20column%20%2211%22%20contains%20%22eur%22 HTTP/1.1" 200 OK
INFO:     77.247.126.189:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     77.247.126.189:0 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [4568]


KeyboardInterrupt: 